# Load libraries 

In [1]:
import numpy as np
import pandas as pd
import jieba
from gensim import corpora
from datetime import datetime

# Input data

In [2]:
data = pd.read_excel('訓練資料.xlsx')

In [3]:
data.head()

,實體一,實體二,關係標記,第二層關係標記,句子
0,慶富,簡良鑑,受聘,工作(公司-人),自稱學界出身的簡良鑑任慶富執行長，幾乎主導慶富大小事，到底在慶富詐貸案扮演何種角色，有待釐清
1,慶富,李瑞倉,合作,NaN,藍：李瑞倉致電華南銀要求給慶富融資10億
2,慶富,陳偉志,其他(涉案),其他,持續追蹤慶富案的立委黃國昌在臉書上表示，無論是何人、基於何目的爆出錄音檔，目前既然出現陳慶男...
3,慶富,慶富,同名,同名,「慶富取得獵雷艦案，從在國防部招標過程，就開始出問題」，陳永興指出，為了讓慶富得標，評選過程...
4,慶富,王端仁,其他(涉案),其他,新頭殼newtalk慶富獵雷艦案延燒高雄市政府，逼得市府海洋局長王端仁請辭，市長陳菊與市議會...


# Set Jieba

In [4]:
jieba.set_dictionary('Dictionary/' + 'dict.txt.big')

In [5]:
add_dic = ['edu_dict.txt', 'company_dict.txt', 'geo_dict.txt', 'law_dict.txt']
for add in add_dic:
    add_path = 'Dictionary/' + add
    dict_df = pd.read_table(add_path, header=None, encoding = 'utf8')
    dict_df['word'] = dict_df.iloc[: ,0].apply(lambda x:x.split()[0])
    dict_df['weight'] = dict_df.iloc[: ,0].apply(lambda x:x.split()[1])
    dict_df = dict_df[['word', 'weight']].values
    for row in dict_df:
        jieba.add_word(row[0], row[1])

C:\Users\hedho\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.
Building prefix dict from C:\Users\hedho\Downloads\關係萃取Pipeline\Dictionary\dict.txt.big ...
Loading model from cache C:\WINDOWS\Temp\jieba.uf15ce8acf8e1db4bb00b591747830337.cache
Loading model cost 1.130 seconds.
Prefix dict has been built succesfully.


In [6]:
for word in set(pd.concat([data.實體一, data.實體二])):
    jieba.add_word(word, 10000)

In [7]:
pred=pd.read_excel('訓練資料.xlsx')
pred.reset_index(inplace=True)
for word in set(pd.concat([pred.實體一, pred.實體二])):
    jieba.add_word(word, 10000)
text_pred = pred['句子'].astype(str).apply(lambda x:jieba.lcut(x))

In [8]:
pred.head()

,index,實體一,實體二,關係標記,第二層關係標記,句子
0,0,慶富,簡良鑑,受聘,工作(公司-人),自稱學界出身的簡良鑑任慶富執行長，幾乎主導慶富大小事，到底在慶富詐貸案扮演何種角色，有待釐清
1,1,慶富,李瑞倉,合作,NaN,藍：李瑞倉致電華南銀要求給慶富融資10億
2,2,慶富,陳偉志,其他(涉案),其他,持續追蹤慶富案的立委黃國昌在臉書上表示，無論是何人、基於何目的爆出錄音檔，目前既然出現陳慶男...
3,3,慶富,慶富,同名,同名,「慶富取得獵雷艦案，從在國防部招標過程，就開始出問題」，陳永興指出，為了讓慶富得標，評選過程...
4,4,慶富,王端仁,其他(涉案),其他,新頭殼newtalk慶富獵雷艦案延燒高雄市政府，逼得市府海洋局長王端仁請辭，市長陳菊與市議會...


In [9]:
text_pred

0        [自稱, 學界, 出身, 的, 簡良鑑, 任, 慶富, 執行長, ，, 幾乎, 主導, 慶富...
1               [藍, ：, 李瑞倉, 致電, 華南銀, 要求, 給, 慶富, 融資, 10, 億]
2        [持續, 追蹤, 慶富, 案, 的, 立委, 黃國昌, 在, 臉, 書上, 表示, ，, 無...
3        [「, 慶富, 取得, 獵, 雷艦, 案, ，, 從, 在, 國防部, 招標, 過程, ，,...
4        [新, 頭殼, newtalk, 慶富, 獵, 雷艦, 案, 延燒, 高雄, 市政府, ，,...
5        [高雄地檢署, 偵辦, 慶富, 造獵, 雷艦, 涉詐, 貸案, ，, 爆疑, 涉喬, 興達,...
6        [標案, 至今, ，, 慶富, 頻遭, 財務, 不佳, 、, 違約, 採購, 中國, 製造,...
7        [檢方, 初步, 將, 全案, 朝涉, 貪汙, 治罪, 條例, 圖利, 罪, 、, 違反, ...
8        [尤其, 慶富, 案, 這個, 標案, ，, 詐貸, 、, 銀行, 聯貸, 通過, 成立, ...
9        [承攬, 海軍, 獵, 雷艦, 案, 的, 慶富造船, 爆發, 嚴重, 財務, 問題, ，,...
10       [財政部, 與, 公股, 行庫, 聯手, 推, 「, 三方, 合約, 」, 特偵組, 若, ...
11       [執行長, 僅涉, 詐, 貸求, 刑, 較輕, 雄檢認, 為, ，, 陳慶男, 、, 簡良鑑...
12       [執行長, 僅涉, 詐, 貸求, 刑, 較輕, 雄檢認, 為, ，, 陳慶男, 、, 簡良鑑...
13       [執行長, 僅涉, 詐, 貸求, 刑, 較輕, 雄檢認, 為, ，, 陳慶男, 、, 簡良鑑...
14       [陳慶男, 父子, 坦承, 涉案, ，, 檢方, 認定, ，, 陳慶男, 等人, 向, 聯貸...
15       [聯合報, -, 慶富, 詐, 貸案, 起訴, 5, 人, 陳慶男, 求刑, 30, 年, ...
16       [(, 工商時報, ), （, 更新, ：, 動, 新聞, ）, 高雄地檢署, 偵辦, 慶富.

# Data preparation

In [10]:
text=data.句子.dropna().apply(lambda x:jieba.lcut(x))

In [11]:
dic = corpora.Dictionary(text)

In [12]:
word2ids = dic.token2id
ids2word = dict([(i,w) for w, i in word2ids.items()])

In [13]:
dic.add_documents(text_pred)
word2ids = dic.token2id
ids2word = dict([(i,w) for w, i in word2ids.items()])

In [14]:
max_len = text.apply(len).max()
n_row = text.shape[0]

word_input = np.zeros(
    (n_row, max_len),
    dtype='float32')

pos_input = np.zeros(
    (n_row, max_len, 2),
    dtype='float32')

In [15]:
for i, row in enumerate(text.tolist()):
    for t, word in enumerate(row):
        word_input[i, t] = word2ids[word]
        if word == data.實體一[i]:
            pos_input[i,t,0] = 1
        elif word == data.實體二[i]:
            pos_input[i,t, 1] = 1

In [16]:
y = data[data.句子.isnull()==False].關係標記

y.loc[y == '兄弟姊妹'] = '親戚'
y.loc[y == '好友'] = '親戚'
y.loc[y == '同門'] = '親戚'
y.loc[y == '父母'] = '親戚'

#y = y.fillna('其他(無關)')
y = y.apply(lambda x:x.split('(')[0].strip())
label_dict = dict([(w,i) for i,w in enumerate(y.unique())])
n_label = len(y.unique())
n_label, y.value_counts()

C:\Users\hedho\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\hedho\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\hedho\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

(17, 無關     2790
 同名     2184
 同盟     1726
 上下游    1723
 受聘      841
 其他      474
 夥伴      390
 同業      364
 異議      148
 合作      140
 共犯      139
 親戚      131
 夫妻      124
 上下級     122
 競爭       88
 告訴       85
 監管       82
 Name: 關係標記, dtype: int64)

In [17]:
from keras.models import Model, Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Input, concatenate, Dropout
from keras.utils import to_categorical

C:\Users\hedho\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\hedho\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hedho\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])

In [18]:
y = to_categorical(y.apply(lambda x:label_dict[x]))

In [19]:
input1 = Input(shape=(max_len, ))
word_embedding = Embedding(input_dim=len(ids2word), output_dim=64, input_length=max_len)(input1)
input2 = Input(shape=(max_len,2))
concat = concatenate([word_embedding, input2])
concat_drop = Dropout(0.5)(concat)
rnn_out = Bidirectional(LSTM(128))(concat_drop)
rnn_drop = Dropout(0.5)(rnn_out)
out = Dense(n_label, activation='softmax')(rnn_drop)

In [20]:
model = Model([input1, input2], out)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [21]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 896)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 896, 64)      1363584     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 896, 2)       0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 896, 66)      0           embedding_1[0][0]                
                                                                 input_2[0][0]                    
__________

# Predict

In [22]:
from keras.models import load_model
model=load_model('model.h5')

In [23]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 896)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 896, 64)      1370432     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 896, 2)       0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 896, 66)      0           embedding_1[0][0]                
                                                                 input_2[0][0]                    
__________

In [24]:
pred=pd.read_excel('測試資料.xlsx')

In [25]:
for word in set(pd.concat([pred.實體一, pred.實體二])):
    jieba.add_word(word, 10000)

In [26]:
text_pred = pred.句子.apply(lambda x:jieba.lcut(x))

In [27]:
dic.add_documents(text_pred)
word2ids = dic.token2id
ids2word = dict([(i,w) for w, i in word2ids.items()])

In [28]:
max_len_pred = text.apply(len).max()
n_row_pred = text_pred.shape[0]

word_input_1 = np.zeros(
    (n_row_pred, max_len_pred),
    dtype='float32')

pos_input_1 = np.zeros(
    (n_row_pred, max_len_pred, 2),
    dtype='float32')

In [29]:
for i, row in enumerate(text_pred.tolist()):
    for t, word in enumerate(row):
        word_input_1[i, t] = word2ids[word]
        if word == pred.實體一[i]:
            pos_input_1[i,t,0] = 1
        elif word == pred.實體二[i]:
            pos_input_1[i,t, 1] = 1

In [30]:
PREDICTION=model.predict([word_input_1,pos_input_1])

In [31]:
inv_label_dict = dict([(w,i) for i,w in label_dict.items()])

In [32]:
pred_label=[inv_label_dict[row] for row in PREDICTION.argmax(axis = 1)]

In [33]:
pred['關係預測']=pred_label

In [34]:
pred[50:100]

,實體一,實體二,句子,關係預測
50,潤泰,潤弘精密,」目前潤泰集團至少擁有6家國內外上市公司，潤泰全球由尹衍樑的夫人王綺帆擔任董座；潤泰創新由簡...,同名
51,潤泰集團,潤泰全,潤泰全(2915)今天舉行股東會，由潤泰集團總裁尹衍樑的妻子、潤泰全董事長王綺帆主持，有小股...,受聘
52,潤泰集團,八德路潤泰集團,潤泰集團總裁尹衍樑先前傳出在家中浴室跌倒、撞傷臉部住院，據媒體報導，尹衍樑在台北榮總住院治療...,無關
53,潤泰集團,大潤發,現在，大潤發、潤泰集團已被社會許多公益團體、慈善機構，視為「災難急救站」,同名
54,潤泰集團,中國大潤發,」目前潤泰集團至少擁有6家國內外上市公司，潤泰全球由尹衍樑的夫人王綺帆擔任董座；潤泰創新由簡...,同名
55,潤泰集團,潤弘精密工程,」目前潤泰集團至少擁有6家國內外上市公司，潤泰全球由尹衍樑的夫人王綺帆擔任董座；潤泰創新由簡...,同名
56,潤泰集團,南山人壽,至於尹衍樑的超級大將、潤泰創新前任董事長劉忠賢，目前主掌匯弘投資擔任董事長，主控潤泰集團轉投...,同名
57,潤泰集團,潤泰精密材料,」目前潤泰集團至少擁有6家國內外上市公司，潤泰全球由尹衍樑的夫人王綺帆擔任董座；潤泰創新由簡...,同名
58,潤泰集團,潤成控股,至於尹衍樑的超級大將、潤泰創新前任董事長劉忠賢，目前主掌匯弘投資擔任董事長，主控潤泰集團轉投...,同名
59,潤泰集團,匯弘投資,至於尹衍樑的超級大將、潤泰創新前任董事長劉忠賢，目前主掌匯弘投資擔任董事長，主控潤泰集團轉投...,同名


In [35]:
pred['關係預測'].value_counts()

同名    88
無關    33
同盟     9
受聘     8
Name: 關係預測, dtype: int64

In [36]:
pred.to_excel('預測結果.xlsx', index = False)